## Train an ML model

Once you have explored the repository and selected an appropriate training dataset, you can download it and train a model. In this notebook we will show an example of how to do so, locally or in the cloud.

## Train a model locally

In order to train a model, you will first need to download it. Here, we will use the [EuroSAT](https://www.eotdl.com/datasets/EuroSAT-RGB) dataset, in its RGB version.

In [13]:
from eotdl.datasets import download_dataset



# download_dataset("EuroSAT-RGB")

This dataset contains 27000 Sentinel 2 images classified in 10 categories. 

In order to streamline the training process, we will use the [PytorchEO](https://github.com/earthpulse/pytorchEO) library. This open source library is a wrapper on top of [Pytorch](https://pytorch.org/) and [Pytorch Lightning](https://lightning.ai/) to facilitate the design, implementation, training and deployment of deep learning models for Earth Observation. It offers AI-Ready EO datasets as well as ready-to-use tasks and models.

In [14]:
import os 

path = 'data/EuroSAT-RGB'

os.listdir(path)

['Industrial',
 'Forest',
 'HerbaceousVegetation',
 'metadata.yml',
 'PermanentCrop',
 'Highway',
 'Residential',
 'SeaLake',
 'River',
 'AnnualCrop',
 'Pasture']

In [15]:
from pytorch_eo.datasets import EuroSATRGB

ds = EuroSATRGB(batch_size=25, verbose=True, path="data", download=False, data_folder = 'EuroSAT-RGB') 

ds.setup()

TypeError: __init__() got an unexpected keyword argument 'data_folder'

In [16]:
import pytorch_eo

pytorch_eo.__version__

AttributeError: module 'pytorch_eo' has no attribute '__version__'